# Advanced Datadriver - SQL

_Execute the following cell in order to make the table of contents appear_

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Overview
In this notebook, we will cover a handy tool to interact with your data directly into your database: the SQL operators

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

# Prerequisite

In order to complete this notebook, you will need: 
* a basic Python knowledge;
* some familiarity with the SQL language;
* access to a Datadriver environment.

You will also need to go over the [Datadriver for DataScientists - Part 1](./0-DatadriverForDataScientists_Part1.ipynb) and [Datadriver for DataScientists - Part 2](./1-DatadriverForDataScientists_Part2.ipynb), if not done yet.

# Setup
As usual, our workflow will contain a few mandatory steps :
* imports
* Instiantiating environment objects
* Instantiating the Context

Once all of this is done, we will load some data and play around with it :

In [ ]:
# Imports
from dd import DB
from dd.api.contexts import LocalContext
from sqlalchemy.exc import ProgrammingError

In [ ]:
# Instiantiating environment objects
db = DB(dbtype='sqlite', filename=':memory:')

In [ ]:
# Instantiating the Context
context = LocalContext(db)

In [ ]:
import pkg_resources
titanic_datapath = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv'
context.load_file(titanic_datapath, table_name='titanic.train').head()

In [ ]:
# Loading some data
train = context.table("titanic.train")

# SQL

## Query

Queries are operations that returns a new Dataset. They may be called thanks to the `Dataset.sql.query()` method. Here's an example on our data :

In [ ]:
select = train.sql.query("select * from `{}` where sex = 'male'".format(train.output_table))

`select` is now a Dataset. You can access it's `head` method:

In [ ]:
select.head()

Once the result has been computed, it is also stored into the database. You can check that it did it's job by using the db object directly to query the database and check that the data is indeed where it's supposed to be :

In [ ]:
db.query('select * from `{}` limit 5'.format(select.output_table))

## Execute

`Dataset.sql.execute()` is similar to `query` because it's is called through the sql property of the dataset, but it does not return a Dataset :

In [ ]:
drop_table = select.sql.execute("drop table if exists `{}`".format(select.output_table))

In [ ]:
drop_table

As you noticed, it returned an action. Meaning all we can do is execute it:

In [ ]:
drop_table()

Thanks to the db object, let's make sure the table was actually deleted :

In [ ]:
try:
    db.query('select * from {} limit 5'.format(select.output_table))
except Exception as error:
    print(error)